# 3. Spark

Spark Programming Guide: <https://spark.apache.org/docs/latest/> (use Python API recommended)
Spark API: <https://spark.apache.org/docs/latest/api/python/index.html>


# 3.1 Example Walkthrough
3.1 Follow the Spark Examples below! After completion see Exercise 3.2 and 3.3!


### Initialize PySpark

First, we use the findspark package to initialize PySpark.

In [2]:
!pip install pyspark

In [3]:
import os, sys

In [4]:
# Initialize PySpark
APP_NAME = "PySpark Lecture"
SPARK_MASTER="local[1]"
import pyspark
import pyspark.sql
from pyspark.sql import Row
conf=pyspark.SparkConf()
conf=pyspark.SparkConf().setAppName(APP_NAME).set("spark.local.dir", os.path.join(os.getcwd(), "tmp"))
sc = pyspark.SparkContext(master=SPARK_MASTER, conf=conf)
spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

print("PySpark initiated...")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 18:26:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/04 18:26:21 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
PySpark initiated...


### Hello, World!

Loading data, mapping it and collecting the records into RAM...

In [8]:
!wget https://raw.githubusercontent.com/scalable-infrastructure/exercise-students-2023/master/data/example.csv

dyld[30313]: Library not loaded: /opt/homebrew/opt/libunistring/lib/libunistring.2.dylib
  Referenced from: <A2808CD2-467C-32EF-B7CF-9441D98ABF5E> /opt/homebrew/Cellar/wget/1.21.3/bin/wget
  Reason: tried: '/opt/homebrew/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/opt/homebrew/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/usr/local/lib/libunistring.2.dylib' (no such file), '/usr/lib/libunistring.2.dylib' (no such file, not in dyld cache), '/opt/homebrew/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/opt/homebrew/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/usr/local/lib/libunistring.2.dylib' (no such file), '/usr/lib/libunistring.2.dylib' (no such file, not in dyld cache)


In [7]:
# Load the text file using the SparkContext
csv_lines = sc.textFile("example.csv")

# Map the data to split the lines into a list
data = csv_lines.map(lambda line: line.split(","))

# Collect the dataset into local RAM
data.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/q224516/Library/CloudStorage/GoogleDrive-andre.luckow@gmail.com/My Drive/02_LMU/VL/2023SS_VL_AAML/code/exercise-students-2023/03_Spark/example.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option

### Creating Objects from CSV

Using a function with a map operation to create objects (dicts) as records...

In [ ]:
# Turn the CSV lines into objects
def csv_to_record(line):
    parts = line.split(",")
    record = {
      "name": parts[0],
      "company": parts[1],
      "title": parts[2]
    }
    return record

# Apply the function to every record
records = csv_lines.map(csv_to_record)

# Inspect the first item in the dataset
records.first()

{'company': 'Relato', 'name': 'Russell Jurney', 'title': 'CEO'}

### GroupBy

Using the groupBy operator to count the number of jobs per person...

In [ ]:
# Group the records by the name of the person
grouped_records = records.groupBy(lambda x: x["name"])

# Show the first group
grouped_records.first()

# Count the groups
job_counts = grouped_records.map(
  lambda x: {
    "name": x[0],
    "job_count": len(x[1])
  }
)

job_counts.first()

job_counts.collect()

[{'job_count': 2, 'name': 'Russell Jurney'},
 {'job_count': 1, 'name': 'Florian Liebert'},
 {'job_count': 1, 'name': 'Don Brown'},
 {'job_count': 1, 'name': 'Steve Jobs'},
 {'job_count': 1, 'name': 'Donald Trump'}]

### Map vs FlatMap

Understanding the difference between the map and flatmap operators...

In [ ]:
# Compute a relation of words by line
words_by_line = csv_lines\
  .map(lambda line: line.split(","))

print(words_by_line.collect())

# Compute a relation of words
flattened_words = csv_lines\
  .map(lambda line: line.split(","))\
  .flatMap(lambda x: x)

flattened_words.collect()

---
## Further Exercises




In [ ]:
!wget https://raw.githubusercontent.com/scalable-infrastructure/exercise-students-2021/master/data/nasa/NASA_access_log_Jul95.gz
!gzip -d NASA_access_log_Jul95.gz

--2021-03-14 10:34:10--  https://raw.githubusercontent.com/scalable-infrastructure/exercise-students-2021/master/data/nasa/NASA_access_log_Jul95.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20676677 (20M) [application/octet-stream]
Saving to: ‘NASA_access_log_Jul95.gz’

NASA_access_log_Jul 100%[===================>]  19.72M  43.8MB/s    in 0.5s    

2021-03-14 10:34:10 (43.8 MB/s) - ‘NASA_access_log_Jul95.gz’ saved [20676677/20676677]

gzip: NASA_access_log_Jul95 already exists; do you wish to overwrite (y or n)? y


3.2 Implement a wordcount using Spark. How many words are in the file `example.csv`?

3.3 Using the NASA Log file, implement a Spark version of the HTTP Response Code Analysis. How many log enteries per HTTP Response Code exist? 